### Imports and setup

In [2]:
import pandas as pd
import numpy as np

time_col_str = 'Time (s)'
latency_col_str = 'Latency (in ms)'
df = pd.read_csv("~/pings/ping_data/output_2024-01-20.csv")

### Analysis

In [ ]:
# Count nan's in the latency column
drops = df[latency_col_str].isnull().sum()

# What's the percentage of ping drops?
drop_rate = drops/len(df)

print(f'There were {drops} drops out of {len(df)} pings, or {drop_rate*100}%')

# Let's do some easy analysis




print(df[latency_col_str].describe())

In [ ]:
# Let's drop the nan's and do some base analytics on the successful pings.
df[latency_col_str] = df[latency_col_str].apply(pd.to_numeric, errors='coerce')
print(df[latency_col_str].describe())


